In [1]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import sklearn
import os
import shap
import keras.backend as K
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
from keras.metrics import top_k_categorical_accuracy
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from skimage.segmentation import mark_boundaries
# Display
from IPython.display import Image

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def top_2_accuracy(in_gt, in_pred):
    return top_k_categorical_accuracy(in_gt, in_pred, k=2)
model = load_model('retinopathy_diabetic/model/full_retina_model.h5',custom_objects={'top_2_accuracy': top_2_accuracy})

/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:304: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
def transform_img_fn(path_list):
    '''Transform image so it can be processed by inception.'''
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(512, 512))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out)

In [4]:
base_image_dir = os.path.join('retinopathy_diabetic', 'figure')
retina_df = pd.read_csv(os.path.join(base_image_dir, 'trainLabels.csv'))
retina_df['PatientId'] = retina_df['image'].map(lambda x: x.split('_')[0])
retina_df['path'] = retina_df['image'].map(lambda x: os.path.join(base_image_dir,
                                                         '{}.jpeg'.format(x)))

In [10]:
retina_df

,image,level,PatientId,path
0,10_left,0,10,retinopathy_diabetic/figure/10_left.jpeg
1,10_right,0,10,retinopathy_diabetic/figure/10_right.jpeg
2,13_left,0,13,retinopathy_diabetic/figure/13_left.jpeg
3,13_right,0,13,retinopathy_diabetic/figure/13_right.jpeg
4,15_left,1,15,retinopathy_diabetic/figure/15_left.jpeg
5,15_right,2,15,retinopathy_diabetic/figure/15_right.jpeg
6,16_left,4,16,retinopathy_diabetic/figure/16_left.jpeg
7,16_right,4,16,retinopathy_diabetic/figure/16_right.jpeg
8,17_left,0,17,retinopathy_diabetic/figure/17_left.jpeg
9,17_right,1,17,retinopathy_diabetic/figure/17_right.jpeg


In [6]:
amostra = np.ndarray(shape=(5, 512, 512,3),
                     dtype=np.float32)

In [10]:
to_explain = transform_img_fn(['retinopathy_diabetic/figure/955_left.jpeg'])

In [8]:
amostra[0] = transform_img_fn(['retinopathy_diabetic/figure/18_left.jpeg'])
amostra[1] = transform_img_fn(['retinopathy_diabetic/figure/1078_left.jpeg'])
amostra[2] = transform_img_fn(['retinopathy_diabetic/figure/1436_left.jpeg'])
amostra[3] = transform_img_fn(['retinopathy_diabetic/figure/1375_right.jpeg'])
amostra[4] = transform_img_fn(['retinopathy_diabetic/figure/158_right.jpeg'])

In [9]:
e = shap.DeepExplainer(model,amostra)

In [11]:
shap_values = e.shap_values(to_explain)

In [14]:
shap.image_plot(shap_values, -to_explain)

ValueError: Floating point image RGB values must be in the 0..1 range.